# Module Imports

In [17]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Formatting

In [18]:
read_file = pd.read_excel ("10_02_2023_Natalie Gorji.xlsx")
read_file.to_csv ("10_02_2023_Natalie Gorji.csv", index = None, header=True)

natalie_all = pd.read_csv('10_02_2023_Natalie Gorji.csv')
natalie_all.drop_duplicates()
natalie_all = natalie_all[:-1]


mikala_numeric = natalie_all.drop(columns=['Date','Opponent','Unnamed: 1'])
mikala_numeric["Chances, %"] = np.where(mikala_numeric["Chances, %"].str.endswith('%'), mikala_numeric["Chances, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges won, %"] = np.where(mikala_numeric["Challenges won, %"].str.endswith('%'), mikala_numeric["Challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in attack won, %"] = np.where(mikala_numeric["Challenges in attack won, %"].str.endswith('%'), mikala_numeric["Challenges in attack won, %"].str.slice(stop=-1), '0')
mikala_numeric["Challenges in defence won, %"] = np.where(mikala_numeric["Challenges in defence won, %"].str.endswith('%'), mikala_numeric["Challenges in defence won, %"].str.slice(stop=-1), '0')
mikala_numeric["Air challenges won, %"] = np.where(mikala_numeric["Air challenges won, %"].str.endswith('%'), mikala_numeric["Air challenges won, %"].str.slice(stop=-1), '0')
mikala_numeric["Successful dribbles, %"] = np.where(mikala_numeric["Successful dribbles, %"].str.endswith('%'), mikala_numeric["Successful dribbles, %"].str.slice(stop=-1), '0')
mikala_numeric["Tackles won, %"] = np.where(mikala_numeric["Tackles won, %"].str.endswith('%'), mikala_numeric["Tackles won, %"].str.slice(stop=-1), '0')

mikala_numeric = mikala_numeric.mask((mikala_numeric == '-'), 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] > mikala_numeric["Score"].str[-1], 100)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] == mikala_numeric["Score"].str[-1], 0)
mikala_numeric["Score"] = mikala_numeric["Score"].mask(mikala_numeric["Score"].str[0] < mikala_numeric["Score"].str[-1], -100)


mikala_numeric = mikala_numeric.apply(pd.to_numeric, errors='ignore')


mikala_numeric


,Score,InStat Index,Position,Goals,Assists,Chances,Chances successful,"Chances, %",Сhances created,Fouls,...,Attacking challenges,"Challenges in attack won, %",Air challenges,"Air challenges won, %",Dribbles,"Successful dribbles, %",Tackles,"Tackles won, %",Ball interceptions,Free ball pick ups
0,-100,141,LM,0,0,0,0,0,0,0,...,7,14,1,0,3,33,3,67,2,6
1,0,138,CM,0,0,1,0,0,0,0,...,10,20,3,0,4,50,5,60,0,5
2,100,174,LM,0,1,1,0,0,2,2,...,11,36,3,67,7,43,2,50,3,2
3,100,135,RM,0,0,0,0,0,0,0,...,7,57,5,80,0,0,7,29,5,4
4,100,136,LM,0,0,0,0,0,0,0,...,9,33,5,60,0,0,3,67,1,1
5,100,143,LM,0,0,0,0,0,0,1,...,7,57,4,75,3,67,1,100,0,1
6,-100,132,LM,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,1,2
7,-100,117,RM,0,0,0,0,0,1,0,...,3,0,3,33,1,0,3,33,3,3
8,100,157,RM,1,0,3,1,33,1,2,...,27,33,4,25,9,22,6,17,5,9
9,100,130,RM,0,0,0,0,0,1,1,...,18,33,4,25,9,22,4,0,5,6


In [19]:
mikala_F = mikala_numeric.loc[mikala_numeric['Position'] == 'F']
mikala_RM = mikala_numeric.loc[mikala_numeric['Position'] == 'RM']
mikala_LM = mikala_numeric.loc[mikala_numeric['Position'] == 'LM']
mikala_CM = mikala_numeric.loc[mikala_numeric['Position'] == 'CM']


In [20]:
# mikala_numeric.corr(method='pearson').round(2)

In [21]:
mikcorr = mikala_numeric.corr(method='spearman').round(2)
# mikcorr.to_excel('Mikala_Correlation.xlsx', engine = 'xlsxwriter')

In [22]:
rounded_F = mikala_F.corr(method='spearman').round(2)

# Print Meaningful Correlations in Top 2 Positions

In [23]:
def important(Dataframe):
    Dataframe_important = Dataframe.iloc[:,0:2]
    Dataframe_important = Dataframe_important.loc[(Dataframe_important['Score'].abs() > 0.4) | (Dataframe_important['InStat Index'].abs() > 0.4)]
    return Dataframe_important

important(rounded_F)

,Score,InStat Index
Score,1.00,0.38
InStat Index,0.38,1.00
Goals,0.35,0.72
Chances,0.33,0.66
Chances successful,0.35,0.72
"Chances, %",0.34,0.71
Сhances created,0.02,0.43
Shots,0.19,0.68
Shots on target,0.28,0.80
xG (Expected goals),0.34,0.75


In [24]:
rounded_RM = mikala_RM.corr(method='spearman').round(2)
important(rounded_RM)

,Score,InStat Index
Score,1.00,0.66
InStat Index,0.66,1.00
Passes,0.43,0.44
Crosses,0.04,0.41
Challenges,0.16,0.42
Attacking challenges,0.20,0.43
"Challenges in attack won, %",0.56,0.50
"Successful dribbles, %",0.49,0.67
"Tackles won, %",-0.41,-0.03
